In [1]:
import pandas as pd

In [2]:
library = pd.read_json("library.json", orient='split')
doc = pd.read_json("doc.json", orient="split").set_index("index")

In [3]:
library.set_index(['videoId'])

,publishedAt,title,source,candidate,source_desc
videoId,,,,,
yy6jBRVDcpI,2020-06-21T01:53:05Z,Vice President Biden Delivers Remarks On Havin...,youtube,Biden,Joe Biden Official YouTube Channel. Playlist: ...
9CBReIcgQYw,2020-06-19T20:42:10Z,"LIVE: President Trump in Tulsa, Oklahoma",youtube,Trump,Donald J Trump Official YouTube Channel. Playl...
-VB24DMVUwY,2020-06-05T03:13:42Z,Join Vice President Biden for A Town Hall with...,youtube,Biden,Joe Biden Official YouTube Channel. Playlist: ...
VYqEo1ehYII,2020-06-02T22:28:33Z,Joe Biden Addresses the Unfolding Situation in...,youtube,Biden,Joe Biden Official YouTube Channel. Playlist: ...
b2aWDG87nJo,2020-06-02T22:23:53Z,Joe Biden Addresses the Nation On the Civil Un...,youtube,Biden,Joe Biden Official YouTube Channel. Playlist: ...
...,...,...,...,...,...
B1iI8AMLGiI,2019-04-08T14:11:19Z,"LIVE: President Trump in Fort Myers, FL",youtube,Trump,Donald J Trump Official YouTube Channel. Playl...
zWICNmMVhLs,2019-04-08T14:11:19Z,"LIVE: President Trump in Lebanon, OH",youtube,Trump,Donald J Trump Official YouTube Channel. Playl...
naiRdSkaVpk,2019-04-08T14:11:19Z,"LIVE: President Trump in Richmond, KY",youtube,Trump,Donald J Trump Official YouTube Channel. Playl...


In [6]:
doc

,text,start,duration,videoId
index,,,,
0,">>Good afternoon,\neveryone.",97.596,2.369,yy6jBRVDcpI
1,I apologize for\nthe slight delay.,99.999,1.501,yy6jBRVDcpI
2,We wanted to make\nsure everyone,101.533,1.335,yy6jBRVDcpI
3,was able to get here.,102.902,2.202,yy6jBRVDcpI
4,Let me think the\nlocal officials,105.137,1.068,yy6jBRVDcpI
...,...,...,...,...
280,we got 1.6 billion dollars we have,1153.830,6.330,Bw0WRGkxi7I
281,another 1.6 million we have a third 1.6,1156.590,5.760,Bw0WRGkxi7I
282,billion coming but we want to build it,1160.160,5.130,Bw0WRGkxi7I


In [ ]:
# example
pd.DataFrame(YouTubeTranscriptApi.get_transcript('yy6jBRVDcpI'))

things to remove:
* (Applause)
* (Music)
* \[Applause\]
* \\n
* \>>

In [12]:
doc.sample(10)

,text,start,duration,videoId
index,,,,
919,one of the gravest threats to the,3407.900,5.160,Px2OG-ren-s
99,government to root out corruption in,332.460,6.960,sL1UF0KDwsE
865,public health briefing and an economic,2351.520,3.180,GXuEUeCH8hg
1032,can't get coded there's a mistrust of,2534.640,3.479,1lbMK2CG63k
166,[Applause],712.670,14.119,nLZCMmrkoZo
267,democracy they're written into our,967.329,6.031,FiPVOx-cAfQ
95,many many decades and on a nuclear way,459.660,7.230,VRwMzHNbzsE
627,should be willing to set up and,2232.390,3.780,xoBgzVWub2A
466,are not able to get accessible,1193.440,7.840,7S4hdImZXdw


In [21]:
# how long are these videos in minutes?
doc[["videoId",'duration']].groupby("videoId").sum()/60
# actually, some of the start times are bizarre. see yy6jBRVDcpI for example - starts at 97.596
# using sum() duration instead

,duration
videoId,
-8KiohxTJ0Y,23.620900
-VB24DMVUwY,59.704533
0Gl_Fl4KvHk,129.981450
0VpB4tllFkk,32.411433
0oX8uWNAgKs,80.711767
...,...
vz4HTSDlQ6g,61.742333
xoBgzVWub2A,91.795383
yGfh4eLgKYw,131.720200


In [22]:
doc.groupby("videoId").agg({'text': lambda x: ' '.join(x)})

,text
videoId,
-8KiohxTJ0Y,this my wife is my sister they switched on me ...
-VB24DMVUwY,">> The country is crying out\nfor leadership, ..."
0Gl_Fl4KvHk,American patriots and I can't wait to be back ...
0VpB4tllFkk,JOE BIDEN: Thanks an awful lot. I know that yo...
0oX8uWNAgKs,welcome everyone to our soul of the nation Sat...
...,...
vz4HTSDlQ6g,welcome everyone to 45 fast my name is Katrina...
xoBgzVWub2A,ladies and gentlemen please welcome to the sta...
yGfh4eLgKYw,down to Houston [Applause] every American it's...


index
0                 >>Good afternoon,\neveryone.
1           I apologize for\nthe slight delay.
2             We wanted to make\nsure everyone
3                        was able to get here.
4            Let me think the\nlocal officials
                        ...                   
280         we got 1.6 billion dollars we have
281    another 1.6 million we have a third 1.6
282     billion coming but we want to build it
283    quickly at one time we don't want to do
284                                   it peace
Name: text, Length: 88094, dtype: object